In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speed up development.

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/content/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development.
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [4]:
# Your code

train = data.sample(frac=0.8, random_state=200)
test = data.drop(train.index)

train.head()
test.head()


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
2,Nora--Cheryl has emailed dozens of memos about...,0
6,"Dear Sir,I am Engr. Ugo Nzego with the Enginee...",1
7,Abedin Huma <AbedinH@state.gov>Saturday Novemb...,0
10,"STRONG><A href=3D""http://www.cnn.com/2003/WORL...",1


## Data Preprocessing

In [5]:
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [6]:
# Your code
# Your code
from bs4 import BeautifulSoup
import re

# remove javascript, css, from my training data and test data
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

# Apply the function to the train and test data
train['text'] = train['text'].apply(remove_html_tags)
test['text'] = test['text'].apply(remove_html_tags)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])

<ipython-input-6-ecfcb959b5fd>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


Dear friend,I am making this contact with you based on trust and confidence I have foryou, your Company and God irrespective of the fact that we have notmet before and because of the nature of the business I am about to introduceto you,I want to remain positive having the faith, the willand convincing myself that you will be interested and capable to do businesswith me.This business proposal I wish to intimate you with isof mutual benefit and its success is entirely based on mutualtrust,cooperation and a high levelof confidentiality as regard thistransaction.My name is Mr. Daniel Johnson The kind of the relationship andthe confidential nature makes it imperative for me to contact youdirectly by email. A German, Late Estate Magnate Mr.Andreas Schranner, amajority stakeholder in Korean Real Estate Company ,until his deathyears ago in 25th July,2000 CONCORDE plane clash [Flight AF4590]http:/ews.bbc.co.uk/1/hi/worl d/europe/859479.stm>),There is a fixeddeposit of US$48,000,000.00(Fourty Ei

<ipython-input-6-ecfcb959b5fd>:8: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [7]:
# Remove html comments
def remove_html_comments(text):
    return re.sub(r"<!--.*?-->", "", text)

# Apply the function to the train and test data
train['text'] = train['text'].apply(remove_html_comments)
test['text'] = test['text'].apply(remove_html_comments)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])

Dear friend,I am making this contact with you based on trust and confidence I have foryou, your Company and God irrespective of the fact that we have notmet before and because of the nature of the business I am about to introduceto you,I want to remain positive having the faith, the willand convincing myself that you will be interested and capable to do businesswith me.This business proposal I wish to intimate you with isof mutual benefit and its success is entirely based on mutualtrust,cooperation and a high levelof confidentiality as regard thistransaction.My name is Mr. Daniel Johnson The kind of the relationship andthe confidential nature makes it imperative for me to contact youdirectly by email. A German, Late Estate Magnate Mr.Andreas Schranner, amajority stakeholder in Korean Real Estate Company ,until his deathyears ago in 25th July,2000 CONCORDE plane clash [Flight AF4590]http:/ews.bbc.co.uk/1/hi/worl d/europe/859479.stm>),There is a fixeddeposit of US$48,000,000.00(Fourty Ei

In [8]:
# Remove html tags
def remove_html_tags(text):
    return re.sub(r"<.*?>", "", text)

# Apply the function to the train and test data
train['text'] = train['text'].apply(remove_html_tags)
test['text'] = test['text'].apply(remove_html_tags)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])

Dear friend,I am making this contact with you based on trust and confidence I have foryou, your Company and God irrespective of the fact that we have notmet before and because of the nature of the business I am about to introduceto you,I want to remain positive having the faith, the willand convincing myself that you will be interested and capable to do businesswith me.This business proposal I wish to intimate you with isof mutual benefit and its success is entirely based on mutualtrust,cooperation and a high levelof confidentiality as regard thistransaction.My name is Mr. Daniel Johnson The kind of the relationship andthe confidential nature makes it imperative for me to contact youdirectly by email. A German, Late Estate Magnate Mr.Andreas Schranner, amajority stakeholder in Korean Real Estate Company ,until his deathyears ago in 25th July,2000 CONCORDE plane clash [Flight AF4590]http:/ews.bbc.co.uk/1/hi/worl d/europe/859479.stm>),There is a fixeddeposit of US$48,000,000.00(Fourty Ei

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters

- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [9]:
# Your code
# remove all special characters
def remove_special_characters(text):
    return re.sub(r"[^a-zA-Z0-9\s]", "", text)

# Apply the function to the train and test data
train['text'] = train['text'].apply(remove_special_characters)
test['text'] = test['text'].apply(remove_special_characters)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])


Dear friendI am making this contact with you based on trust and confidence I have foryou your Company and God irrespective of the fact that we have notmet before and because of the nature of the business I am about to introduceto youI want to remain positive having the faith the willand convincing myself that you will be interested and capable to do businesswith meThis business proposal I wish to intimate you with isof mutual benefit and its success is entirely based on mutualtrustcooperation and a high levelof confidentiality as regard thistransactionMy name is Mr Daniel Johnson The kind of the relationship andthe confidential nature makes it imperative for me to contact youdirectly by email A German Late Estate Magnate MrAndreas Schranner amajority stakeholder in Korean Real Estate Company until his deathyears ago in 25th July2000 CONCORDE plane clash Flight AF4590httpewsbbccouk1hiworl deurope859479stmThere is a fixeddeposit of US4800000000Fourty Eight Million United States Dollars m

In [10]:
#remove all numbers from the text data
def remove_numbers(text):
    return re.sub(r"\d", "", text)

# Apply the function to the train and test data
train['text'] = train['text'].apply(remove_numbers)
test['text'] = test['text'].apply(remove_numbers)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])


Dear friendI am making this contact with you based on trust and confidence I have foryou your Company and God irrespective of the fact that we have notmet before and because of the nature of the business I am about to introduceto youI want to remain positive having the faith the willand convincing myself that you will be interested and capable to do businesswith meThis business proposal I wish to intimate you with isof mutual benefit and its success is entirely based on mutualtrustcooperation and a high levelof confidentiality as regard thistransactionMy name is Mr Daniel Johnson The kind of the relationship andthe confidential nature makes it imperative for me to contact youdirectly by email A German Late Estate Magnate MrAndreas Schranner amajority stakeholder in Korean Real Estate Company until his deathyears ago in th July CONCORDE plane clash Flight AFhttpewsbbccoukhiworl deuropestmThere is a fixeddeposit of USFourty Eight Million United States Dollars madein a Korea bank since  w

In [11]:
#remove all single characters from the text data
def remove_single_characters(text):
    return re.sub(r"\b\w\b", "", text)

# Apply the function to the train and test data
train['text'] = train['text'].apply(remove_single_characters)
test['text'] = test['text'].apply(remove_single_characters)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])

Dear friendI am making this contact with you based on trust and confidence  have foryou your Company and God irrespective of the fact that we have notmet before and because of the nature of the business  am about to introduceto youI want to remain positive having the faith the willand convincing myself that you will be interested and capable to do businesswith meThis business proposal  wish to intimate you with isof mutual benefit and its success is entirely based on mutualtrustcooperation and  high levelof confidentiality as regard thistransactionMy name is Mr Daniel Johnson The kind of the relationship andthe confidential nature makes it imperative for me to contact youdirectly by email  German Late Estate Magnate MrAndreas Schranner amajority stakeholder in Korean Real Estate Company until his deathyears ago in th July CONCORDE plane clash Flight AFhttpewsbbccoukhiworl deuropestmThere is  fixeddeposit of USFourty Eight Million United States Dollars madein  Korea bank since  which is

In [12]:
#Remove single characters from the start
def remove_single_characters(text):
    return re.sub(r"\s+[a-zA-Z]\s+", " ", text)

# Apply the function to the train and test data
train['text'] = train['text'].apply(remove_single_characters)
test['text'] = test['text'].apply(remove_single_characters)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])

Dear friendI am making this contact with you based on trust and confidence  have foryou your Company and God irrespective of the fact that we have notmet before and because of the nature of the business  am about to introduceto youI want to remain positive having the faith the willand convincing myself that you will be interested and capable to do businesswith meThis business proposal  wish to intimate you with isof mutual benefit and its success is entirely based on mutualtrustcooperation and  high levelof confidentiality as regard thistransactionMy name is Mr Daniel Johnson The kind of the relationship andthe confidential nature makes it imperative for me to contact youdirectly by email  German Late Estate Magnate MrAndreas Schranner amajority stakeholder in Korean Real Estate Company until his deathyears ago in th July CONCORDE plane clash Flight AFhttpewsbbccoukhiworl deuropestmThere is  fixeddeposit of USFourty Eight Million United States Dollars madein  Korea bank since  which is

In [13]:
#Substitute multiple spaces with single space
def remove_extra_spaces(text):
    return re.sub(r"\s+", " ", text)

# Apply the function to the train and test data
train['text'] = train['text'].apply(remove_extra_spaces)
test['text'] = test['text'].apply(remove_extra_spaces)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])


Dear friendI am making this contact with you based on trust and confidence have foryou your Company and God irrespective of the fact that we have notmet before and because of the nature of the business am about to introduceto youI want to remain positive having the faith the willand convincing myself that you will be interested and capable to do businesswith meThis business proposal wish to intimate you with isof mutual benefit and its success is entirely based on mutualtrustcooperation and high levelof confidentiality as regard thistransactionMy name is Mr Daniel Johnson The kind of the relationship andthe confidential nature makes it imperative for me to contact youdirectly by email German Late Estate Magnate MrAndreas Schranner amajority stakeholder in Korean Real Estate Company until his deathyears ago in th July CONCORDE plane clash Flight AFhttpewsbbccoukhiworl deuropestmThere is fixeddeposit of USFourty Eight Million United States Dollars madein Korea bank since which is eight y

In [14]:
#Remove prefixed 'b'
def remove_prefixed_b(text):
    return re.sub(r"^b\s+", "", text)

# Apply the function to the train and test data
train['text'] = train['text'].apply(remove_prefixed_b)
test['text'] = test['text'].apply(remove_prefixed_b)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])

Dear friendI am making this contact with you based on trust and confidence have foryou your Company and God irrespective of the fact that we have notmet before and because of the nature of the business am about to introduceto youI want to remain positive having the faith the willand convincing myself that you will be interested and capable to do businesswith meThis business proposal wish to intimate you with isof mutual benefit and its success is entirely based on mutualtrustcooperation and high levelof confidentiality as regard thistransactionMy name is Mr Daniel Johnson The kind of the relationship andthe confidential nature makes it imperative for me to contact youdirectly by email German Late Estate Magnate MrAndreas Schranner amajority stakeholder in Korean Real Estate Company until his deathyears ago in th July CONCORDE plane clash Flight AFhttpewsbbccoukhiworl deuropestmThere is fixeddeposit of USFourty Eight Million United States Dollars madein Korea bank since which is eight y

In [15]:
#Convert to Lowercase
def to_lowercase(text):
    return text.lower()

# Apply the function to the train and test data
train['text'] = train['text'].apply(to_lowercase)
test['text'] = test['text'].apply(to_lowercase)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])

dear friendi am making this contact with you based on trust and confidence have foryou your company and god irrespective of the fact that we have notmet before and because of the nature of the business am about to introduceto youi want to remain positive having the faith the willand convincing myself that you will be interested and capable to do businesswith methis business proposal wish to intimate you with isof mutual benefit and its success is entirely based on mutualtrustcooperation and high levelof confidentiality as regard thistransactionmy name is mr daniel johnson the kind of the relationship andthe confidential nature makes it imperative for me to contact youdirectly by email german late estate magnate mrandreas schranner amajority stakeholder in korean real estate company until his deathyears ago in th july concorde plane clash flight afhttpewsbbccoukhiworl deuropestmthere is fixeddeposit of usfourty eight million united states dollars madein korea bank since which is eight y

## Now let's work on removing stopwords
Remove the stopwords.

In [16]:
# Your code
def remove_stopwords(text):
    words = text.split()
    words = [word for word in words if word not in stopwords.words("english")]
    return " ".join(words)

# Apply the function to the train and test data
train['text'] = train['text'].apply(remove_stopwords)
test['text'] = test['text'].apply(remove_stopwords)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])


dear friendi making contact based trust confidence foryou company god irrespective fact notmet nature business introduceto youi want remain positive faith willand convincing interested capable businesswith methis business proposal wish intimate isof mutual benefit success entirely based mutualtrustcooperation high levelof confidentiality regard thistransactionmy name mr daniel johnson kind relationship andthe confidential nature makes imperative contact youdirectly email german late estate magnate mrandreas schranner amajority stakeholder korean real estate company deathyears ago th july concorde plane clash flight afhttpewsbbccoukhiworl deuropestmthere fixeddeposit usfourty eight million united states dollars madein korea bank since eight years sincethenno claim withdrawal made depositfortunately person charge deposit dayof deposit date great late estate magnate died without awritten oral attached account respect hisinvestment korea real estate company perceivedpossibility able locate

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [17]:
# Your code
#lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# break sentences into words using lemmatization
def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

# Apply the function to the train and test data
train['text'] = train['text'].apply(lemmatize_text)
test['text'] = test['text'].apply(lemmatize_text)

# Check if the function works
print(train['text'].iloc[0])
print(test['text'].iloc[0])


[nltk_data] Downloading package wordnet to /root/nltk_data...


dear friendi making contact based trust confidence foryou company god irrespective fact notmet nature business introduceto youi want remain positive faith willand convincing interested capable businesswith methis business proposal wish intimate isof mutual benefit success entirely based mutualtrustcooperation high levelof confidentiality regard thistransactionmy name mr daniel johnson kind relationship andthe confidential nature make imperative contact youdirectly email german late estate magnate mrandreas schranner amajority stakeholder korean real estate company deathyears ago th july concorde plane clash flight afhttpewsbbccoukhiworl deuropestmthere fixeddeposit usfourty eight million united state dollar madein korea bank since eight year sincethenno claim withdrawal made depositfortunately person charge deposit dayof deposit date great late estate magnate died without awritten oral attached account respect hisinvestment korea real estate company perceivedpossibility able locate nex

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [18]:
# Your code


## Extra features

In [19]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x))

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x))

data_train.head()

NameError: name 'data_train' is not defined

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code